In [1]:
import os
import time
import pycuda.autoinit

from utils.yolo_classes import get_cls_dict
from utils.display import open_window, set_display, show_fps
from utils.visualization import BBoxVisualization
from utils.yolo import TRT_YOLO

trt_yolo = TRT_YOLO("yolov4-tiny-416",(416,416),5)

In [ ]:
import cv2
from matplotlib.pyplot import imshow
%matplotlib inline

img = cv2.imread('3.jpg')

boxes, confs, clss = trt_yolo.detect(img,0.3)
for i in range(len(clss)):
    cv2.rectangle(img,(boxes[i][0],boxes[i][1]),(boxes[i][2],boxes[i][3]),(0,0,255),5)
print(clss)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
imshow(img)

[2.]


In [ ]:
camera.start()

In [3]:
import traitlets
from IPython.display import display

import ipywidgets.widgets as widgets
from jetbot import Camera
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
#from jetcam.usb_camera import USBCamera
#from jetcam.utils import bgr8_to_jpeg

camera = Camera.instance(width=416,height=416)
image = widgets.Image(format='jpeg',width=224,height=224)
camera_link = traitlets.dlink((camera,'value'),(image,'value'),transform = bgr8_to_jpeg)
robot = Robot()

In [4]:
display(image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [5]:
def modifySpeed(origin_road):
    global speed_left,speed_right,speed
    
    averaged_lines = getAverageLines(origin_road)
    
    if averaged_lines is not None:
        mid_x = int(np.average(averaged_lines , axis =0)[2])
            
        if mid_x < 315:
            speed_left = speed
            speed_right = speed + 0.03
        elif mid_x > 325:
            speed_left = speed
            speed_right = speed + 0.03
        else:
            speed_left = speed +0.01
            speed_right = speed

In [6]:
import time

stop_ignore = 0
railway_ignore = 0
count = 0

def getNearest(signs):
    if not len(signs):
        return [0,3]
    t = time.time()
    
    if signs[0][1] == 4 and (t>railway_ignore)\
        or signs[0][1] in [0,1,2,3]:
            return signs[0]
    return getNearest(signs[1:]) if len(signs) >= 2 else [0,3]

def getConfs(signs, confs):
    if not len(signs):
        return [0, 5]
    
    return signs[confs.tolist().index(max(confs.tolist()))]

In [64]:
def update(change):
    global robot ,count,speed_left,speed_right,stop_ingore,railway_ingore,stop,trigger
    ALERT_WIDTH = 50
    img = change['new']
    #modifySpeed(img)
    
    if count < 10:
        count += 1
        return
    else:
        count = 0
    boxes,confs,clss = trt_yolo.detect(img,0.3)
    
    signs = []
    for box,cls in zip(boxes,clss):
        width = box[2] - box[0]
        signs.append([width,cls])
    signs.sort(reverse=True,key = lambda x :x[0])
    sign = getNearest(signs)
    #sign = getConfs(signs, confs)
    
    t = time.time()
    if sign[1] == 3 and statu[0] == True :
        trigger[0] = True
        statu[0] = False
        robot.set_motors(speed_left * 1.5,speed_right*1.5)
        time.sleep(1)
        robot.set_motors(speed_left,speed_right)
    elif sign[1] == 1 and statu[1] == True:
        trigger[0] = True
        robot.set_motors(speed_left * 0.5,speed_right*0.5)
        statu[1] = False
        time.sleep(1)
        robot.set_motors(speed_left,speed_right)
    elif sign[1] == 2 and statu[2] == True:
        trigger[0] = True
        robot.set_motors(speed_left * 0.8,speed_right*-0.8)
        time.sleep(1)
        robot.set_motors(speed_left ,speed_right)
        statu[2] = False
        
    if sign[1] == 0  and statu[3] == True:
        robot.stop()
        stop = False
        statu[3] = False
    elif sign[1] == 4 and statu[4] == True:
        trigger[3] = True
        robot.stop()
        time.sleep(5)
        robot.set_motors(speed_left,speed_right)
        statu[4] = False
    else:
        robot.set_motors(speed_left,speed_right)

In [ ]:
trigger = [False for i in range(4)]
speed = speed_left =0.174
speed_right = 0.16
stop = True
statu = [True,True,True,True,True]

robot.set_motors(speed_left,speed_right)
while(stop):
    update({'new':camera.value})

In [70]:
from jetbot import Robot
speed = speed_left = speed_right = 0
robot = Robot()
robot.stop()

In [71]:
camera_link.unlink()
camera.stop()

In [ ]:
robot.stop()

In [ ]:
stop=False

In [ ]:
img = camera.value
boxes,confs,clss = trt_yolo.detect(img,0.3)

averaged_lines = getAverageLines(img)[0]
if averaged_lines is not None:
    for x1,y1,x2,y2 in averaged_lines:
        cv2.line(img,(int(x1/640*224),int(y1/480*224)),(int(x2/640*224),int(y2/480*224)),(0,0,255),5)
    mid_x = int(np.average(averaged_lines , axis =0)[2])
    cv2.line(img,(int(mid_x/640*224),100),(int(mid_x/640*224),105),(0,255,0),2)
    print("mid: ",mid_x)
for i in range(len(clss)):
    cv2.rectangle(img,(boxes[i][0],boxes[i][1]),(boxes[i][2],boxes[i][3]),(0,0,255),1)
img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
imshow(img)